In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import pandas as pd

In [ ]:
!which chromedriver

In [18]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/abby/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [19]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [20]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [22]:
# Create empty lists for scraped data to be stored in.
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []
reviewURL_list = []


listOfPages = soup.find('ul', class_='paginator-list').find_all('li')
# print(listOfPages)
numberOfPages = listOfPages[0].find('a')['aria-label'].split()[3]
print(numberOfPages)

for i in numberOfPages:

    today = datetime.datetime.now()

    for product in products:

        # Get the link to the product page.
        link = product.find('a')
        href = link['href']
        product_url = base_url + href
        productURL_list.append(product_url)

        # Get the link to the product image.
        img = product.find('img')['src']
        productImage_list.append(img)

        # Get the product title.
        product_title = product.find('img')['alt']
        productTitle_list.append(product_title)

        # Try to get the number of reviews, there may not be any.
        try:
            review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
            reviewAmount_list.append(review_amount)

        except:
            review_amount = 0
            reviewAmount_list.append(review_amount)

        # In the case where there are at least one review...
        if int(review_amount) > 0:
            # Get the average number of stars
            stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
            starReview_list.append(stars_review)

            # Get the URL to the reviews section for that product.
            review_url = product.find('div', class_='stars').find('a')['href']
            reviewURL_list.append(base_url+review_url)

        # Otherwise, use defalt 0 or NaN values for these entries.
        else:
            stars_review = 0
            starReview_list.append(stars_review)

            review_url = "NaN"
            reviewURL_list.append(review_url)

        # Get the price of the product.
        price = product.find('span', class_='price-main')
        current_price = price.find('span', class_='visuallyhidden').text
        currentPrice_list.append(current_price)

    # Close the browser window
    # browser.quit()

    # Create a dictionary with the lists of the scrapped data.
    data = {
        "Title": productTitle_list,
        "URL": productURL_list,
        "Image": productImage_list,
        "AverageStars": starReview_list,
        "NumberofReviews": reviewAmount_list,
        "ReviewsURL": reviewURL_list,
        "Price": currentPrice_list
           }

    # Create a Pandas DataFrame with that dictionary
    product_df = pd.DataFrame.from_dict(data)
    product_df.head()

,Title,URL,Image,AverageStars,NumberofReviews,ReviewsURL,Price
0,High-Efficiency HEPA-Type Desktop Air Purifier...,https://www.walmart.com/ip/High-Efficiency-HEP...,https://i5.walmartimages.com/asr/5ba9740e-a3f1...,4.4,136,https://www.walmart.com/ip/High-Efficiency-HEP...,$52.30
1,Filtrete F2 Bacteria & Virus True HEPA Room Ai...,https://www.walmart.com/ip/Filtrete-F2-Bacteri...,https://i5.walmartimages.com/asr/dea94bac-be28...,4.8,55,https://www.walmart.com/ip/Filtrete-F2-Bacteri...,$17.97
2,"Filtrete by 3M Room Air Purifier, Console, 110...",https://www.walmart.com/ip/Filtrete-by-3M-Room...,https://i5.walmartimages.com/asr/eae28c2f-5dbe...,4.6,195,https://www.walmart.com/ip/Filtrete-by-3M-Room...,$49.94
3,Air Purifier for Home Large Room with HEPA Fil...,https://www.walmart.com/ip/Air-Purifier-Home-L...,https://i5.walmartimages.com/asr/ea3ecf29-543f...,3.3,3,https://www.walmart.com/ip/Air-Purifier-Home-L...,$88.88
4,Airfree T800 Filterless Virus Destroying Air P...,https://www.walmart.com/ip/Airfree-T800-Filter...,https://i5.walmartimages.com/asr/a6f53c0f-31d5...,0,0,NaN,$99.00


In [23]:
product_df = product_df.astype({
    "NumberofReviews": 'int',
    "AverageStars": 'float'
})

product_df.dtypes

Title               object
URL                 object
Image               object
AverageStars       float64
NumberofReviews      int64
ReviewsURL          object
Price               object
dtype: object

In [24]:
product_df.describe()

,AverageStars,NumberofReviews
count,40.000000,40.000000
mean,3.582500,85.125000
std,1.734401,149.415474
min,0.000000,0.000000
25%,3.825000,2.750000
50%,4.400000,19.000000
75%,4.500000,103.500000
max,5.000000,712.000000


In [42]:
# nextbutton = soup.find('button', class_='paginator-btn')
# nextbutton = browser.find_by_name('2').first.click()
# browser.visit(nextbutton)
listOfPages = soup.find('ul', class_='paginator-list').find_all('li')
# print(listOfPages)
numberOfPages = listOfPages[0].find('a')['aria-label'].split()[3]
print(numberOfPages)
# page 1 &amp;cat_id=0&amp;grid=true&amp;ps=40
# page 2 &amp;page=2&amp;cat_id=0&amp;grid=true&amp;ps=40
# page 3 &amp;page=3&amp;cat_id=0&amp;grid=true&amp;ps=40

13


In [12]:
# Get reviews 

review_df = pd.DataFrame()

# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,6):
    if product_df['NumberofReviews'][i] > 0:
        
        productTitleList = []
        reviewURLList = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
        
        if len(reviewRatingList) < product_df['NumberofReviews'][i]:
#         while len(reviewRatingList) < product_df['NumberofReviews'][i]:
    
            try:
                browser.visit(product_df["ReviewsURL"][i])
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
                browser.visit(base_url+allReviewsURL)
                soup = BeautifulSoup(html, 'html.parser')
                reviews = soup.find_all('div', class_='ReviewList-content')

                # need to add a way to make sure I am capturing all of the reviews
                # can check this by looking at NumberofReview for this ReviewsRUL
                # to scroll down a little use the below command
                # browser.execute_script("window.scrollTo(200, document.body.scrollHeight);")
                # check the average rating by using the values found here



                for review in reviews:

                    productTitleList.append(product_df['Title'][i])
                    reviewURLList.append(product_df['ReviewsURL'][i])

                    try:
                        title = review.find('h3', class_='review-title').text
                        reviewTitleList.append(title)
                        print(title)

                    except:
                        title = 'None'
                        reviewTitleList.append(title)
                        print(title)
#                         pass

                    # adding try only because the browser needs to scroll
                    # once this is added, there wont need to be a try
                    # each review has to have a star value
                    try:
                        reviewRating = review.find('span', class_='seo-avg-rating').text
                        reviewRatingList.append(reviewRating)
                        print(reviewRating)
                    except:
                        pass

                    # adding try and except for the review body for same reason as stars
                    try:
                        reviewComment = review.find('p').text
                        reviewCommentList.append(reviewComment)
#                         print(reviewComment)
                    except:
                        pass
                
            
                # only until I fix the scroll
                reviewTitleList.pop()
                productTitleList.pop()
                reviewURLList.pop()
                
                data = {
                    'ProductTitle': productTitleList,
                    'ReviewURL': reviewURLList,
                    'ReviewTitle': reviewTitleList,
                    'ReviewStarRating': reviewRatingList,
                    'ReviewComment': reviewCommentList
                }
                print('stored data')
                productReviews_df = pd.DataFrame.from_dict(data)
                productReviews_df.head()
                print('into dataframe')

            except:
                pass
    
        print('add to existing dataframe')
        review_df = review_df.append(productReviews_df, ignore_index=True)
print('done')
#                 browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

add to existing dataframe


NameError: name 'productReviews_df' is not defined

In [ ]:
# len(review_df)
# review_df.tail()
review_df.describe()

In [ ]:
reviews = soup.find('div', class_='review-highlight')
positive = reviews.find('div', class_='font-bold highlight-title').text
stars = reviews.find('span', class_='seo-avg-rating').text
# starts = reviews.find('span', class_='seo-average-rating')
body = reviews.find('div', class_='collapsable-content-container').text
print(positive)
print(stars)
print(body)

In [ ]:
reviews = soup.find_all('div', class_='ReviewList-content')

In [13]:
# Get reviews 

review_df = pd.DataFrame()

# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,5):
    if product_df['NumberofReviews'][i] > 0:
#     number_of_reviews = product_df['NumberofReviews'][i]
#     for j in range(number_of_reviews):
        
        productTitleList = []
        reviewURLList = []
        reviewSiteLink = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
        
#         if len(reviewRatingList) < product_df['NumberofReviews'][i]:
        
#         while len(reviewRatingList) < product_df['NumberofReviews'][i]:
    
        try:
            # go to the product url link
            browser.visit(product_df["ReviewsURL"][i])
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the 'see all reviews' button and go there
            allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
            browser.visit(base_url+allReviewsURL)
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the list of reviews
            reviews = soup.find_all('div', class_='ReviewList-content')
            
#             for review in reviews:
            for j in range(len(reviews)):
                reviewSiteLink.append(review)
                reviewSiteLink[j].send_keys(Keys.PAGE_DOWN)
                soup = BeautifulSoup(html, 'html.parser')
                reviews = soup.find('div', class_='ReviewList-content')
                
#                 print(f"j={j}")
                productTitleList.append(product_df['Title'][i])
                reviewURLList.append(product_df['ReviewsURL'][i])
                title, stars, comments = scrapeReviews(review)
                reviewTitleList.append(title)
                reviewRatingList.append(stars)
                reviewCommentList.append(comments)


                data = {
                    'ProductTitle': productTitleList,
                    'ReviewURL': reviewURLList,
                    'ReviewTitle': reviewTitleList,
                    'ReviewStarRating': reviewRatingList,
                    'ReviewComment': reviewCommentList
                }
                print('stored data')
                productReviews_df = pd.DataFrame.from_dict(data)
                productReviews_df.head()
                print('into dataframe')

        except:
            pass

        print('add to existing dataframe')
        review_df = review_df.append(productReviews_df, ignore_index=True)
print('done')

add to existing dataframe


NameError: name 'productReviews_df' is not defined

In [ ]:
# len(review_df)
# review_df.tail()
review_df.describe()

In [ ]:
def scrapeReviews(review):
    
    # get the title of the review
    try:
        title = review.find('h3', class_='review-title').text
    except:
        title = 'None'
        
    # get the number of stars in this review
    reviewRating = review.find('span', class_='seo-avg-rating').text
    
    # get the context of the review
    reviewComment = review.find('p').text
        
    # return these items as a tuple
    return title, stars, comments

In [9]:
browser.quit()

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")